In [ ]:
rvs = stats.bernoulli(p=0.2).rvs
bern = random(N, N, density=0.2, random_state=rs, data_rvs=rvs)
bern.A

In [ ]:
rvs = stats.poisson(25, loc=10).rvs
poisson = random(N, N, density=0.1, random_state=rs, data_rvs=rvs)
poisson.A

In [1]:
from scipy.sparse import random
from scipy.sparse import csr_matrix
from numpy.linalg import inv as inv
from numpy.linalg import norm as norm
from scipy import stats
import numpy as np
import time

In [2]:
class CustomRandomState(object):
    def randint(self, k):
        i = np.random.randint(k)
        return i - i % 2
    

In [3]:
def check_abs_eignevalues_less_than_1(data):
    ''' Checks if abs of max eigenvalue is less than 1 
    '''
#     make data at each dimesion be centered around mean
    data -= data.mean(axis=0)
#     get covariance matrix
    Sigma = np.cov(data, rowvar=False)
#     find eigenvalues
    eigvals = np.linalg.eigvals(Sigma)
    return max(abs(max(eigvals)),abs(min(eigvals))) < 1

In [4]:
def vonNeumanApproximation(I,B,n):
    '''Returns von Neuman approimation in form I+B+B^2+B^3+..+B^n
    '''
    result = I
    B_power = B
    for i in range(1,n):
#         add current power of B to result
        result += B_power
#         increase power of B
        B_power = B_power @ B
    return result

In [14]:
rs = CustomRandomState()
N = 1000
n = 3
ro = 0.01

In [15]:
dense_normal = np.random.normal(loc = 0, scale = 1/(N*ro), size=(N, N))
sparse_normal = csr_matrix(dense_normal)

In [16]:
dense_bernoulli = np.random.binomial(n=1, p=ro, size=(N, N))
sparse_bernoulli = csr_matrix(dense_bernoulli)

In [17]:
B = np.multiply(sparse_normal , sparse_bernoulli)

In [18]:
check_abs_eignevalues_less_than_1(B)

False

In [19]:
I_minus_B = np.identity(N) - B

In [20]:
t1 = time.time()
real_inverse = inv(I_minus_B)
t2 = time.time()
print("caluclated invserse in",(t2-t1)/60,"mins")
print('approximated NEUMAN inverse by',n,'approximations')
neuman_appr = vonNeumanApproximation(np.identity(N), B, n)
t3 = time.time()

print("caluclated vonneuman in",(t3-t2)/60,"mins")

print('error in neuman approximation in form norm(real_inverse-approx_inverse)')
print(norm(real_inverse - neuman_appr))
print('error in random')
print(norm(real_inverse - np.random.rand(N,N)))

caluclated invserse in 0.0018797477086385092 mins
approximated NEUMAN inverse by 3 approximations
caluclated vonneuman in 0.1108344833056132 mins
error in neuman approximation in form norm(real_inverse-approx_inverse)
3185.6480596726565
error in random
663.4002418583493
